In [1]:
import os
from typing import Dict, List, Any, Optional, Sequence, TypedDict, Annotated
import operator
from dotenv import load_dotenv

from langchain_core.messages import BaseMessage, SystemMessage, HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_community.tools import WikipediaQueryRun, YouTubeSearchTool, DuckDuckGoSearchResults
from langchain_community.tools import TavilySearchResults
from langchain_community.utilities import WikipediaAPIWrapper
from langchain_openai import ChatOpenAI

load_dotenv()  # Load environment variables from .env file

# Set up LLM
DEFAULT_MODEL = os.getenv("DEFAULT_MODEL", "gpt-3.5-turbo")

def get_model(model_name=DEFAULT_MODEL):
    """Get a chat model instance with the specified model name."""
    return ChatOpenAI(model=model_name, api_key=os.getenv("OPENAI_API_KEY"), temperature=0.0)

# Define common tools
def get_common_tools():
    """Get a list of common tools for agents."""
    wikipedia_tool = WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper())
    youtube_tool = YouTubeSearchTool()
    # web_search_tool = DuckDuckGoSearchResults()
    web_search_tool = TavilySearchResults(api_key=os.getenv("TAVILY_API_KEY"))

    return [wikipedia_tool, youtube_tool, web_search_tool]

def get_code_execution_tools():
    """Get a list of tools for isolated code execution with Daytona."""
    try:
        from daytona_tools import get_daytona_tools
        return get_daytona_tools()
    except ImportError:
        print("Warning: daytona_tools module not found. Code execution tools unavailable.")
        return []

def get_filesystem_tools():
    """Get a list of tools for filesystem operations and planning."""
    try:
        from filesystem_planner_agent import get_filesystem_tools as get_fs_tools
        return get_fs_tools()
    except ImportError:
        print("Warning: filesystem_planner_agent module not found. Filesystem tools unavailable.")
        return []

# Base agent state (used in most architectures)
class AgentState(TypedDict):
    """Base state that includes conversation messages."""
    messages: Annotated[Sequence[BaseMessage], operator.add]

# Extended agent state for more complex architectures
class ExtendedAgentState(AgentState):
    """Extended state with additional fields for complex architectures."""
    intermediate_steps: List[Dict[str, Any]]
    selected_agents: List[str]
    current_agent_idx: int
    final_response: Optional[str]

# Common system prompts
RESEARCHER_PROMPT = """You are a skilled research agent that specializes in finding information.
When given a question, break it down and search for relevant facts and data.
Provide detailed, factual responses and cite your sources when possible."""

WRITER_PROMPT = """You are a skilled writing agent that excels at clear communication.
Your goal is to take information and transform it into well-structured, coherent text.
Focus on clarity, organization, and making complex information accessible."""

CRITIC_PROMPT = """You are a critical thinking agent that evaluates information.
Your job is to review content, identify potential issues, logical fallacies, or missing information.
Provide constructive feedback aimed at improving accuracy and quality."""

PLANNER_PROMPT = """You are a strategic planning agent that helps organize complex tasks.
Given a problem, break it down into clear, actionable steps.
Consider dependencies between tasks and create an efficient plan of action."""

INTEGRATION_PROMPT = """You are an integration agent that synthesizes information from multiple sources.
Combine diverse inputs into a coherent whole, identifying connections and resolving contradictions.
Present a unified perspective that represents the combined knowledge."""

CODE_EXECUTOR_PROMPT = """You are an advanced code execution agent that specializes in running code in secure, isolated environments using Daytona sandboxes.

Your capabilities include:
- Creating and managing Daytona sandboxes for isolated code execution
- Executing Python code and shell commands safely in sandboxed environments
- Managing files within sandboxes (upload, download, create, modify)
- Cloning Git repositories into sandboxes
- Providing detailed execution results with stdout, stderr, and exit codes

Key principles:
1. ALWAYS create a sandbox before executing any code
2. Provide clear feedback about sandbox status and operations
3. Handle errors gracefully and provide helpful debugging information
4. Clean up resources when tasks are complete
5. Ensure code execution is secure and isolated from the host system

When a user asks you to run code:
1. First check if a sandbox exists, create one if needed
2. Execute the code in the isolated environment
3. Return the results with clear formatting
4. Offer to save any generated files or outputs

Available tools:
- create_daytona_sandbox: Create a new isolated sandbox
- destroy_daytona_sandbox: Remove a sandbox when done
- execute_python_code: Run Python code in the sandbox
- execute_command_in_sandbox: Run shell commands in the sandbox
- upload_file_to_sandbox: Upload files to the sandbox
- download_file_from_sandbox: Download files from the sandbox
- list_daytona_sandboxes: List available sandboxes
- git_clone_in_sandbox: Clone repositories into the sandbox

Always prioritize security and isolation. Never execute potentially harmful code on the host system."""

FILESYSTEM_PLANNER_PROMPT = """You are an advanced filesystem planner agent that specializes in creating, managing, and executing structured plans using todo.md files and filesystem operations.

Your enhanced capabilities include:
- Creating and managing todo.md files with structured plans
- Performing filesystem operations (create directories, read/write files, etc.)
- Tracking task progress and updating plans dynamically
- Working with both actual files and state-based plan management
- Organizing tasks with priorities and dependencies

Key workflow:
1. FIRST: Create a structured todo.md plan for the given task
2. Break down complex tasks into manageable subtasks
3. Execute tasks one by one while updating the plan
4. Mark completed tasks and track progress
5. Refine and reorganize the plan as needed

Available tools:
- filesystem_operation: Create directories, read/write files, list contents
- todo_manager: Create, update, and manage todo.md plans
- plan_updater: Refine plans, add subtasks, reorganize priorities

Plan structure guidelines:
- Use clear task descriptions with checkboxes
- Include overview and notes sections
- Track completion status
- Add timestamps for updates
- Organize by priority and dependencies

Always start by creating a comprehensive plan, then execute it systematically while keeping the plan updated.

You are a strategic planning agent that helps organize complex tasks.
Given a problem, break it down into clear, actionable steps.
Consider dependencies between tasks and create an efficient plan of action."""

# Helper function to create a basic agent prompt template
def create_agent_prompt(system_message: str) -> ChatPromptTemplate:
    """Create a standard agent prompt template with the given system message."""
    return ChatPromptTemplate.from_messages([
        SystemMessage(
            content=system_message,
        ),
        MessagesPlaceholder(variable_name="chat_history", optional=True),
    ])


In [2]:
"""
Enhanced Human-in-the-Loop Agent

A more robust implementation of a human-in-the-loop workflow where the agent can request human input
during execution and incorporate human feedback using advanced LangGraph interrupts.
"""

import asyncio
from typing import Dict, List, Any, Annotated, Sequence, TypedDict, Optional, Literal, Union
import operator

from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, PromptTemplate
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from langgraph.types import interrupt, Command




"""
Enhanced Human-in-the-Loop Agent with Improved Workflow

A robust implementation focusing on a dedicated Human-in-the-Loop node 
with advanced routing and feedback mechanisms.
"""

import asyncio
import nest_asyncio

from typing import Dict, List, Any, Annotated, Sequence, TypedDict, Optional, Literal
import operator

from langchain_core.messages import BaseMessage, HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder, PromptTemplate
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
from langgraph.prebuilt import create_react_agent
from langgraph.types import interrupt, Command
nest_asyncio.apply()


# Define the state for the human-in-the-loop workflow
class HumanInTheLoopState(TypedDict):
    """State for human-in-the-loop workflow."""
    messages: Annotated[Sequence[BaseMessage], operator.add]
    human_input_needed: bool
    human_input_reason: str
    human_input_response: Optional[str]
    internal_thoughts: List[str]
    confidence: float
    feedback_on_plan: Optional[str]
    next_node: Optional[str]  # New field to track routing


async def initialize_state(state: HumanInTheLoopState) -> Dict:
    """Initialize the state for the workflow."""
    return {
        "human_input_needed": False,
        "human_input_reason": "",
        "human_input_response": None,
        "internal_thoughts": [],
        "confidence": 1.0,
        "feedback_on_plan": None,
        "next_node": None  # Initialize next_node
    }


async def task_planner(state: HumanInTheLoopState) -> Dict:
    """Plan the approach to solving the task and prepare for human feedback."""
    query = state["messages"][-1].content
    
    # Create the planner
    llm = get_model()
    tools = get_common_tools()
    
    # Define a specific prompt for task planning
    task_planning_prompt = ChatPromptTemplate.from_messages([
        ("system", """You are an advanced task planning assistant using web search and other 
        available tools to create a comprehensive and well-researched plan.

        Your goal is to:
        1. Analyze the user's request thoroughly
        2. Use available tools to gather relevant information
        3. Create a detailed, step-by-step plan
        4. Identify potential challenges and areas needing human input
        5. Provide rationale for each step of the plan

        When creating the plan, consider:
        - Breaking down complex tasks into manageable steps
        - Using tool-gathered information to inform your planning
        - Highlighting areas that may require human expertise or input
        """),
       ("placeholder", "{messages}"),
    ])
    
    # Partial the prompt with tools and tool names
    task_planning_prompt = task_planning_prompt.partial(
        tools="\n".join([f"- {tool.name}: {tool.description}" for tool in tools]),
        tool_names=", ".join([tool.name for tool in tools])
    )

    # Create the React agent
    agent = create_react_agent(
        llm, 
        tools, 
        prompt=task_planning_prompt
    )
    
    response = agent.invoke({
        "messages": query
    })
    
    plan = response["messages"][-1].content
    
    print(f"Generated Plan:\n{plan}")
    # planner_prompt = PromptTemplate(
    #     template="""You are a task planning assistant. Your job is to analyze the user's request
    #     and create a step-by-step plan for addressing it.
        
    #     User request: {query}
        
    #     Create a detailed plan with 3-5 steps for completing this task. For each step:
    #     1. Describe what needs to be done
    #     2. Identify potential challenges or areas needing human input
    #     3. Explain the rationale behind each step
        
    #     Format your response as a structured, clear plan.
    #     """,
    #     input_variables=["query"]
    # )
    
    
    # # Generate the plan
    # chain = planner_prompt | llm
    # response = chain.invoke({"query": query})
    # plan = response.content
    
    return {
        "internal_thoughts": [f"Initial Task Plan: {plan}"],
        "human_input_needed": True,  # Trigger human input node
        "human_input_reason": f"Please review the following task plan:\n\n{plan}\n\nDo you approve this plan?",
        "next_node": None  # Will be set by human feedback node
    }


async def human_feedback_node(state: HumanInTheLoopState) -> Dict:
    """
    Dedicated node for handling human feedback and routing.
    
    This node:
    1. Presents the plan or request for human review
    2. Captures human feedback
    3. Determines the next routing based on feedback
    """
    llm = get_model()
    
    # Use interrupt to get human input
    feedback_request = state.get("human_input_reason", "No specific reason provided.")
    
    # Request human input with routing options
    human_response = interrupt(f"""
    {feedback_request}

    Possible actions:
    1. Approve and continue to next step
    2. Request modifications
    3. Cancel or restart

    Please respond with:
    - '1' to approve and continue
    - '2' followed by your suggestions for modification
    - '3' to cancel or restart
    """)
    
    # Process human feedback
    if isinstance(human_response, str):
        response_type = human_response[0]
        
        if response_type == '1':
            # Approved, route to confidence assessment
            return {
                "human_input_needed": False,
                "human_input_response": "Plan approved",
                "feedback_on_plan": "Approved by human",
                "next_node": "assess_confidence",
                "internal_thoughts": state["internal_thoughts"] + ["Human approved the plan"]
            }
        
        elif response_type == '2':
            # Modifications requested
            modification_details = human_response[1:].strip()
            
            # Use LLM to revise plan based on human feedback
            revision_prompt = PromptTemplate(
                template="""You are a task planning assistant. Revise the original plan 
                based on the following human feedback:
                
                Original Plan: {original_plan}
                
                Human Feedback: {human_feedback}
                
                Provide a revised plan addressing the feedback while maintaining 
                the core objectives of the original plan.
                """,
                input_variables=["original_plan", "human_feedback"]
            )
            
            original_plan = state["internal_thoughts"][-1].replace("Initial Task Plan: ", "")
            
            # Generate revised plan
            revision_chain = revision_prompt | llm
            revised_response = revision_chain.invoke({
                "original_plan": original_plan,
                "human_feedback": modification_details
            })
            
            return {
                "human_input_needed": True,
                "human_input_reason": f"Revised Plan:\n\n{revised_response.content}\n\nDo you approve this revised plan?",
                "internal_thoughts": state["internal_thoughts"] + [
                    f"Human Modification Request: {modification_details}",
                    f"Revised Task Plan: {revised_response.content}"
                ],
                "next_node": None  # Will trigger another human feedback iteration
            }
        
        elif response_type == '3':
            # Cancel or restart
            return {
                "human_input_needed": False,
                "human_input_response": "Process cancelled by human",
                "feedback_on_plan": "Cancelled",
                "next_node": END,  # Or another appropriate node
                "internal_thoughts": state["internal_thoughts"] + ["Process cancelled by human"]
            }
        
    # Fallback for unexpected input
    return {
        "human_input_needed": False,
        "human_input_response": "Unexpected input",
        "next_node": END,
        "internal_thoughts": state["internal_thoughts"] + ["Unexpected human input received"]
    }


async def assess_confidence(state: HumanInTheLoopState) -> Dict:
    """Assess confidence level and determine if additional human input is needed."""
    query = state["messages"][-1].content
    llm = get_model()
    
    confidence_prompt = PromptTemplate(
        template="""Assess the confidence in executing the following task plan:
        
        Task: {query}
        
        Current Plan: {task_plan}
        
        Rate your confidence from 0.0 to 1.0. If confidence is below 0.7:
        1. Explain specific areas of uncertainty
        2. Recommend what additional information or input would help
        
        Output format:
        Confidence: [0.0-1.0]
        Uncertainty Areas: [list of specific concerns]
        Additional Input Needed: [detailed recommendations]
        """,
        input_variables=["query", "task_plan"]
    )
    
    # Extract the most recent task plan from internal thoughts
    task_plans = [
        thought.replace("Initial Task Plan: ", "").replace("Revised Task Plan: ", "") 
        for thought in state["internal_thoughts"] 
        if "Task Plan:" in thought
    ]
    task_plan = task_plans[-1] if task_plans else "No detailed plan available"
    
    # Generate confidence assessment
    chain = confidence_prompt | llm
    response = chain.invoke({
        "query": query,
        "task_plan": task_plan
    })
    
    # Parse the response
    confidence = 1.0
    uncertainty_areas = ""
    additional_input = ""
    
    for line in response.content.split("\n"):
        if line.startswith("Confidence:"):
            try:
                confidence = float(line.split(":")[1].strip())
            except:
                confidence = 1.0
        elif line.startswith("Uncertainty Areas:"):
            uncertainty_areas = line.split(":")[1].strip()
        elif line.startswith("Additional Input Needed:"):
            additional_input = line.split(":")[1].strip()
    
    # Determine next steps based on confidence
    if confidence < 0.7:
        return {
            "confidence": confidence,
            "human_input_needed": True,
            "human_input_reason": f"""
            Confidence is low for the current task plan.
            
            Uncertainty Areas: {uncertainty_areas}
            
            Additional Input Needed: {additional_input}
            
            Please provide more context or guidance.
            """,
            "internal_thoughts": state["internal_thoughts"] + [
                f"Confidence Assessment: {response.content}",
                f"Confidence Level: {confidence}"
            ],
            "next_node": None  # Will trigger human feedback node
        }
    else:
        return {
            "confidence": confidence,
            "human_input_needed": False,
            "next_node": "generate_response",
            "internal_thoughts": state["internal_thoughts"] + [
                f"Confidence Assessment: {response.content}",
                f"Confidence Level: {confidence}"
            ]
        }


async def generate_response(state: HumanInTheLoopState) -> Dict:
    """Generate a response based on the task and any human input."""
    query = state["messages"][-1].content
    
    # Create the agent
    llm = get_model()
    tools = get_common_tools()
    
    PROMPT = """You are an assistant that works collaboratively with humans.
    Incorporate any human feedback or input received during the planning process.
    
    Provide a comprehensive and thoughtful response that addresses the original query
    while reflecting on any guidance or modifications suggested during the workflow.
    """
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", PROMPT),
        MessagesPlaceholder(variable_name="chat_history", optional=True),
    ])
    
    # Create the agent
    agent = create_react_agent(llm, tools, prompt=prompt)
    
    # Execute the agent
    response = await agent.ainvoke({
        "input": query,
        "chat_history": state.get("messages", [])
    })
    
    # Final human review of the response
    review_request = f"""
    I've prepared a response to your request. Please review:

    {response["messages"][-1].content}

    Actions:
    1. Accept the response
    2. Request modifications
    3. Restart the process

    Respond with:
    - '1' to accept
    - '2' followed by modification requests
    - '3' to restart
    """
    
    review_response = interrupt(review_request)
    
    # Process review response
    if isinstance(review_response, str):
        response_type = review_response[0]
        
        if response_type == '1':
            # Response accepted
            return {
                "messages": [response["messages"][-1]],
                "next_node": END,
                "internal_thoughts": state.get("internal_thoughts", []) + ["Final response accepted by human"]
            }
        
        elif response_type == '2':
            # Modifications requested
            modification_details = review_response[1:].strip()
            
            # Revise response
            revision_prompt = PromptTemplate(
                template="""Revise the response based on human feedback:
                
                Original Query: {query}
                Original Response: {original_response}
                
                Human Modification Request: {modification_request}
                
                Provide a modified response addressing the specific feedback.
                """,
                input_variables=["query", "original_response", "modification_request"]
            )
            
            revision_chain = revision_prompt | llm
            revised_response = revision_chain.invoke({
                "query": query,
                "original_response": response["messages"][-1].content,
                "modification_request": modification_details
            })
            
            return {
                "messages": [AIMessage(content=revised_response.content)],
                "next_node": END,
                "internal_thoughts": state.get("internal_thoughts", []) + [
                    "Response modification requested",
                    f"Modification details: {modification_details}",
                    f"Revised response: {revised_response.content}"
                ]
            }
        
        elif response_type == '3':
            # Restart process
            return {
                "messages": [],
                "next_node": START,
                "internal_thoughts": state.get("internal_thoughts", []) + ["Process restarted at user's request"]
            }
    
    # Fallback
    return {
        "messages": [response["messages"][-1]],
        "next_node": END,
        "internal_thoughts": state.get("internal_thoughts", []) + ["Response processed with default handling"]
    }


def route_next_node(state: HumanInTheLoopState) -> str:
    """
    Route to the next node based on the state's next_node attribute.
    Provides a flexible routing mechanism.
    """
    next_node = state.get("next_node")
    
    # Default routing logic
    if next_node:
        return next_node
    elif state.get("human_input_needed"):
        return "human_feedback"
    else:
        return "generate_response"


def create_human_in_the_loop_workflow():
    """Create and return the human-in-the-loop workflow."""
    # Create the workflow graph
    workflow = StateGraph(HumanInTheLoopState)
    
    # Add nodes
    workflow.add_node("initialize", initialize_state)
    workflow.add_node("task_planner", task_planner)
    workflow.add_node("human_feedback", human_feedback_node)
    workflow.add_node("assess_confidence", assess_confidence)
    workflow.add_node("generate_response", generate_response)
    
    # Add edges
    workflow.add_edge(START, "initialize")
    workflow.add_edge("initialize", "task_planner")
    
    # Add conditional edges with more flexible routing
    workflow.add_conditional_edges(
        "task_planner",
        route_next_node,
        {
            "human_feedback": "human_feedback",
            "assess_confidence": "assess_confidence",
            "generate_response": "generate_response",
            END: END
        }
    )
    
    workflow.add_conditional_edges(
        "human_feedback",
        route_next_node,
        {
            "task_planner": "task_planner",
            "assess_confidence": "assess_confidence",
            "generate_response": "generate_response",
            END: END
        }
    )
    
    workflow.add_conditional_edges(
            "assess_confidence",
            route_next_node,
            {
                "human_feedback": "human_feedback",
                "generate_response": "generate_response",
                END: END
            }
        )
    
    workflow.add_conditional_edges(
        "generate_response",
        route_next_node,
        {
            END: END
        }
    )
    
    # Set the default end point
    workflow.add_edge("generate_response", END)
    
    # Compile the workflow
    return workflow.compile(checkpointer=MemorySaver())


async def run_human_in_the_loop_example(query: str, simulate_human_input: Dict[str, str] = None):
    """
    Run an example query through the human-in-the-loop workflow.
    
    Args:
        query: The user query to process
        simulate_human_input: Dictionary mapping interrupt points to simulated responses
    """
    # Create a custom interrupt handler
    from langchain_core.callbacks import CallbackManager
    from langchain_core.callbacks.base import BaseCallbackHandler

    class SimpleInterruptHandler(BaseCallbackHandler):
        def __init__(self, simulate_inputs=None):
            self.simulate_inputs = simulate_inputs or {}
            self.interrupt_count = 0
        
        async def on_interrupt(self, message, run_id, **kwargs):
            """Handle interrupt by simulating user input or requesting from console."""
            self.interrupt_count += 1
            print(f"\nInterrupt #{self.interrupt_count} triggered: {message}")
            
            if self.simulate_inputs:
                # Priority-based simulation
                simulation_keys = [
                    f'interrupt_{self.interrupt_count}', 
                    'default_response'
                ]
                
                for key in simulation_keys:
                    if key in self.simulate_inputs:
                        response = self.simulate_inputs[key]
                        print(f"Simulated response for {key}: {response}")
                        return response
            
            # Fallback to console input
            print(f"\nHuman input requested: {message}")
            user_input = input("Your response: ")
            return user_input
    
    # Initialize the workflow
    workflow = create_human_in_the_loop_workflow()
    
    # Initialize the state with the query
    initial_state = {
        "messages": [HumanMessage(content=query)]
    }
    
    # Create the interrupt handler with simulated inputs
    interrupt_handler = SimpleInterruptHandler(simulate_inputs=simulate_human_input)
    
    # Set up the config with our interrupt handler
    config = {
        "configurable": {"thread_id": "1"},
        "callbacks": [interrupt_handler]
    }
    
    print(f"\n=== Processing query: {query} ===")
    
    try:
        # Execute the workflow
        result = await workflow.ainvoke(initial_state, config)
        
        # Extract final results
        final_response = "No response generated."
        if result.get("messages"):
            # Get the last AI message
            for msg in reversed(result["messages"]):
                if isinstance(msg, AIMessage):
                    final_response = msg.content
                    break
                    
        internal_thoughts = result.get("internal_thoughts", [])
        confidence = result.get("confidence", 0.0)
        human_input_needed = result.get("human_input_needed", False)
        human_input_reason = result.get("human_input_reason", "")
        human_input_response = result.get("human_input_response", "")
        
        return {
            "final_response": final_response,
            "internal_thoughts": internal_thoughts,
            "confidence": confidence,
            "human_input_needed": human_input_needed,
            "human_input_reason": human_input_reason,
            "human_input_response": human_input_response
        }
        
    except Exception as e:
        print(f"Error during workflow execution: {e}")
        return {
            "final_response": f"Error: {str(e)}",
            "internal_thoughts": [],
            "confidence": 0.0,
            "human_input_needed": False,
            "human_input_reason": "",
            "human_input_response": ""
        }


if __name__ == "__main__":
    import asyncio
    
    # Example queries with simulated inputs
    queries = [
        {
            "query": "Create a comprehensive workout plan for improving overall fitness.",
            "simulated_input": {
                "interrupt_1": "2Add more emphasis on cardiovascular exercises",
                "interrupt_2": "1",  # Accept the response
                "default_response": "1"  # Fallback for other interrupts
            }
        },
        {
            "query": "Help me plan a trip to Japan for two weeks.",
            "simulated_input": {
                "interrupt_1": "2Include more cultural experiences and local cuisine recommendations",
                "interrupt_2": "1",  # Accept the response
                "default_response": "1"  # Fallback for other interrupts
            }
        }
    ]
    
    # Run examples
    async def run_examples():
        print("\n=== Human-in-the-Loop Agent Results ===")
        
        for i, example in enumerate(queries):
            print(f"\n--- Example {i+1} ---")
            print(f"Query: {example['query']}")
            
            result = await run_human_in_the_loop_example(
                example["query"], 
                simulate_human_input=example["simulated_input"]
            )
            
            print(f"\nConfidence: {result['confidence']}")
            print(f"Human Input Needed: {result['human_input_needed']}")
            if result['human_input_needed']:
                print(f"Human Input Reason: {result['human_input_reason']}")
                print(f"Human Input Response: {result['human_input_response']}")
            
            print("\nFinal Response:")
            print(result["final_response"])
            
            print("\nInternal Thoughts (for debugging):")
            for thought in result["internal_thoughts"]:
                print(f"- {thought}")
    
    # Run the examples
    asyncio.run(run_examples())


=== Human-in-the-Loop Agent Results ===

--- Example 1 ---
Query: Create a comprehensive workout plan for improving overall fitness.

=== Processing query: Create a comprehensive workout plan for improving overall fitness. ===
Generated Plan:
To create a comprehensive workout plan for improving overall fitness, we need to consider a balanced combination of cardiovascular exercises, strength training, flexibility exercises, and rest days for recovery. Here is a detailed step-by-step plan:

1. **Assess Current Fitness Level**:
   - Evaluate your current fitness level, including strength, endurance, flexibility, and cardiovascular fitness.
   - Consider any medical conditions or physical limitations that may impact your workout plan.

2. **Set Realistic Goals**:
   - Define specific and achievable fitness goals such as weight loss, muscle gain, improved endurance, or flexibility.
   - Break down long-term goals into smaller, manageable milestones.

3. **Cardiovascular Exercises**:
   - I